# Cython

* Cython is one of python's *dialects* to bridge between C and python.

* Following code block is an [example](https://stackoverflow.com/questions/35656604/running-cython-in-jupyter-ipython) of the `cython` code.

In [ ]:
%load_ext Cython



In [ ]:
%%cython -a

def geo_prog_cython(double alpha, int n):
    cdef double current = 1.0
    cdef double sum = current
    cdef int i
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum



In [ ]:
%%time

geo_prog_cython(0.5, 5)



In [ ]:
def geo_prog_python(alpha, n):
    current = 1.0
    sum = current
    
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum



In [ ]:
%%time

geo_prog_python(0.5, 5)



## Visualizing

* `matplotlib` can visualize results from `cython` functions.

In [ ]:
%load_ext Cython



In [ ]:
%%cython --annotate

def diff_eq(xi):

    # dx/dt + x = 0
    return -xi


def euler_cython():

    # initial values
    cdef double ti = 0.0
    cdef double te = 10.0
    cdef double delta_t = 1e-3
    cdef double x0 = 1.0
    
    cdef double t = ti
    
    cdef int n = int((te - ti) / delta_t) + 1
    cdef int i = 0
    
    # https://stackoverflow.com/questions/25974975/cython-c-array-initialization
    cdef double result_t[10001]
    result_t = [0.0] * 10001
    cdef double result_x[10001]
    result_x = [0.0] * 10001
    
    cdef double dx_dt = 0
    
    result_t[0] = ti
    result_x[0] = x0
    
    for i in range(1, n):
        dx_dt = diff_eq(result_x[i-1])
        result_x[i] = result_x[i-1] + dx_dt * delta_t
        result_t[i] = result_t[i-1] + delta_t

    return result_t, result_x



In [ ]:
def diff_eq_python(xi):

    # dx/dt + x = 0
    return -xi


def euler_python():

    # initial values
    ti = 0.0
    te = 10.0
    delta_t = 1e-3
    x0 = 1.0
    
    t = ti
    
    n = int((te - ti) / delta_t) + 1
    i = 0
    
    result_t = [0.0] * 10001
    result_x = [0.0] * 10001
    
    result_t[0] = ti
    result_x[0] = x0
    
    dx_dt = 0
    
    for i in range(1, n):
        dx_dt = diff_eq_python(result_x[i-1])
        result_x[i] = result_x[i-1] + dx_dt * delta_t
        result_t[i] = result_t[i-1] + delta_t

    return result_t, result_x



In [ ]:
%%time
# measure time to calculate

t, x = euler_cython()



In [ ]:
%%time
# measure time to calculate

t_py, x_py = euler_python()



In [ ]:
import pylab as py


py.plot(t, x, 'o', label='cython')
py.plot(t_py, x_py, '-', label='python')
py.grid(True)
py.xlabel('t')
py.ylabel('y')
py.legend(loc=0)
py.show()



## Calling C/C++ functions

* Cython can call C/C++ functions as follows.
[[ref0](https://cython.readthedocs.io/en/latest/src/userguide/external_C_code.html)]
, [[ref1](https://stackoverflow.com/questions/37426534/how-can-i-import-an-external-c-function-into-an-ipython-notebook-using-cython)]
, [[ref2](https://stackoverflow.com/questions/19260253/cython-compiling-error-multiple-definition-of-functions)]
, [[ref3](https://media.readthedocs.org/pdf/cython/stable/cython.pdf)]
, [[ref4](http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html)]
 
 

### With `numpy` support

* If we need to use matrices and vectors frequently, combining `numpy` and cython may be helpful.

* Following is file `cos_cython_numpy.h`.

In [ ]:
%%writefile cos_cython_numpy.h
/*
    2.8.5.2. Numpy Support, 2.8.5. Cython, http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13
*/
void cos_cython_numpy_func(double * in_array, double * out_array, int size);



* Following is file `cos_cython_numpy.c`.

In [ ]:
%%writefile cos_cython_numpy.c
/*
2.8.5.2. Numpy Support, 2.8.5. Cython, http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13
*/

#include <math.h>

/*  Compute the cosine of each element in in_array, storing the result in
 *  out_array. */
void cos_cython_numpy_func(double * in_array, double * out_array, int size){
    int i;
    for(i=0;i<size;i++){
        out_array[i] = cos(in_array[i]);
    }
}



* Following is file `_cos_cython_numpy.pyx`.

In [ ]:
%%writefile _cos_cython_numpy.pyx
""" Example of wrapping a C function that takes C double arrays as input using
    the Numpy declarations from Cython 
    Valentin Haenel, 2.8.5.2. Numpy Support, 2.8.5. Cython, Scipy Lectures, Oct 18 2016, [Online] 
        Available: http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13 
"""

""" Example of wrapping a C function that takes C double arrays as input using
    the Numpy declarations from Cython """

# cimport the Cython declarations for numpy
cimport numpy as np

# if you want to use the Numpy-C-API from Cython
# (not strictly necessary for this example, but good practice)
np.import_array()

# cdefine the signature of our c function
cdef extern from "cos_cython_numpy.h":
    void cos_cython_numpy_func (double * in_array, double * out_array, int size)

# create the wrapper code, with numpy type annotations
def cos_cython_numpy_py_func(np.ndarray[double, ndim=1, mode="c"] in_array not None,
                     np.ndarray[double, ndim=1, mode="c"] out_array not None):
    cos_cython_numpy_func(<double*> np.PyArray_DATA(in_array),
                <double*> np.PyArray_DATA(out_array),
                in_array.shape[0])



* Following is file `setup.py`.

In [ ]:
%%writefile setup.py

# Valentin Haenel, 2.8. Interfacing with C,  Scipy Lectures, Oct 18 2016, [Online]
#   Available: http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html

from distutils.core import setup, Extension

import numpy
from Cython.Distutils import build_ext

print('for NumPy Support of Cython '.ljust(60, '#'))
setup(cmdclass={'build_ext': build_ext},
      ext_modules=[Extension("cos_cython_numpy",
                             sources=['_cos_cython_numpy.pyx', "cos_cython_numpy.c"],
                             include_dirs=[numpy.get_include()])],
      )



In [ ]:
%%bash
python setup.py build_ext --inplace



In [ ]:
import pylab as py
import cos_cython_numpy

x = py.arange(0, 2 * py.pi, 0.1)
y = py.empty_like(x)

cos_cython_numpy.cos_cython_numpy_py_func(x, y)
py.plot(x, y)
py.show()



In [ ]:
%%bash
rm cos_cython_numpy.h
rm cos_cython_numpy.c
rm _cos_cython_numpy.c
rm _cos_cython_numpy.pyx
rm setup.py



In [ ]:
assert (y == py.cos(x)).all(), "Cython Result != Pylab Expected"

